<a href="https://colab.research.google.com/github/chetools/CVE2005_Spring2025/blob/main/PipeNetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
from scipy.optimize import newton, root

In [2]:
rho = 1e3
mu = 1e-3
g = 9.81

def churchill(Re, eD):
    t1 =(-2.457*np.log((7/Re)**0.9+0.27*eD)   )**16
    t2 = (37530/Re)**16
    return 8* (  (8/Re)**12 + 1/(t1+t2)**1.5  )**(1/12)

In [37]:
#from   to  L   D   eD

pipes = np.array([[0, 1, 200., 0.1, 1e-4],
                  [1, 2, 100., 0.1, 1e-4],
                  [1, 3, 150., 0.1, 1e-4],
                  [1, 4, 50., 0.1, 1e-4],
                  [2, 4, 5., 0.1, 1e-4],
                  [2, 3, 80., 0.1, 1e-4],
                  [3, 4, 10., 0.1, 1e-4]])

node_z=np.array([30., -5., 3., 5., 0.])

node_pressures = np.array([3e5, np.nan, np.nan, np.nan, 0. ])
node_heads = node_pressures/rho/g + node_z
from_nodes = pipes[:,0].astype(int)
to_nodes = pipes[:,1].astype(int)
pipe_L = pipes[:,2]
pipe_D = pipes[:,3]
pipe_eD = pipes[:,4]



In [38]:
internal_idx=np.where(np.isnan(node_heads))[0]
external_idx=np.where(np.logical_not(np.isnan(node_heads)))[0]
node_heads[internal_idx] = np.mean(node_heads[external_idx])

In [28]:
def calc_hdrop(L, D, eD, Q):
    V = np.abs(4*Q/np.pi/D**2)
    Re = rho*V*D/mu
    f= churchill(Re, eD)
    return np.copysign(f*L/D * (V**2)/2/g, Q)

In [29]:
def flow(L, D, eD, hdrop):

    hdrop_abs = np.abs(hdrop)
    if hdrop_abs<1e-15:
        return 0.
    Qguess= np.sqrt(2*g*D*hdrop_abs/(0.05*L)) * np.pi*(D**2)/4
    def fun(Q):
        return calc_hdrop(L,D, eD, Q) - hdrop_abs
    return np.copysign(newton(fun, Qguess), hdrop)

In [54]:
def calc_pipeflows(node_heads):
    pipeflows = np.zeros_like(pipe_D)
    for i, (L, D, eD, from_node, to_node) in enumerate(zip(pipe_L,pipe_D,pipe_eD,from_nodes,to_nodes)):
        h_from = node_heads[from_node]
        h_to = node_heads[to_node]
        hdrop = h_from - h_to
        pipeflows[i]= flow(L, D, eD, hdrop)
    return pipeflows

In [55]:
def internal_netflows(x):
    node_heads[internal_idx] = x
    pipeflows = calc_pipeflows(node_heads)
    netflows=np.zeros_like(node_z)
    for i, pipeflow in enumerate(pipeflows):
        np.add.at(netflows, from_nodes[i], -pipeflow)
        np.add.at(netflows, to_nodes[i], pipeflow)
    return netflows[internal_idx]

In [56]:
res=root(internal_netflows, np.full(internal_idx.size, np.mean(node_heads[external_idx])))
res.x

array([3.25532828, 0.16919923, 0.18152106])

In [57]:
node_heads[internal_idx] = res.x
calc_pipeflows(node_heads)

array([ 0.04959415,  0.01504115,  0.01203528,  0.02251772,  0.01581248,
       -0.00077133,  0.01126395])

In [58]:
node_heads

array([60.58103976,  3.25532828,  0.16919923,  0.18152106,  0.        ])

In [60]:
P

array([300000.        ,  80984.77041138, -27770.15554267, -47269.27839147,
            0.        ])